2022년 7월까지의 데이터를 학습하고 8,9,10월의 데이터를 예측할것

In [207]:
import pandas as pd
import numpy as np
from haversine import haversine
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow import keras
import numpy as np
from numpy import concatenate
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
import random
import itertools
from sklearn import linear_model

함수정의

In [208]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
      cols.append(df.shift(i))
      names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
      cols.append(df.shift(-i))
      if i == 0:
          names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
      else:
          names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
      agg.dropna(inplace=True)
  return agg

In [209]:
#기준 아파트로부터 'near_range' km 내에 존재하는 아파트 코드 반환
def finding_near_apt(gizun,near_range):
    start = (float(gizun['GPS_Y']),float(gizun['GPS_X']))
    near_apts = []
    for j in df_range.index:
        goal = (float(df_range['GPS_Y'][j]),float(df_range['GPS_X'][j]))
        if haversine(start, goal) < near_range:
            near_apts.append(df_range['apart_code'][j])
    near_df = pd.DataFrame()
    near_df['apart_code'] = near_apts
    return near_df

In [210]:
#사용하지 않을 과거 데이터 제거(윈도우 사이즈가 3일경우)
def delete_unuse_var(reframed):
    var_count = int(len(reframed.columns)/4)
    reframed.drop(reframed.columns[var_count*2:(var_count*3) - 1], axis=1, inplace=True)
    reframed.drop(reframed.columns[var_count:(var_count*2) - 1], axis=1, inplace=True)
    reframed.drop(reframed.columns[0:var_count - 1], axis=1, inplace=True)
    return reframed

In [211]:
#데이터프레임에서 일정한 컬럼만 추출하고 값 형태로 반환
def get_float_values(df):
    df = df[['YYYYMM','sale_type', 'area_m3', 'floor', 'const_year', 
    'parking','crime_rate', 'building_year', 'market_dist','busstop_dist', 'subway_dist', 
    'kinder_dist', 'hospital_dist', 'ASPI','YYYY', 'MM', 'HAI', 'PIR', 'LIR', 'GDP', 'GDI', 
    'GNI', 'CPI','avg_interest', 'kospi', 'M2', 'house_CCSI', 'unsold','near_element', 
    'near_middle', 'near_high', 'generation','const_rank','prices']]
    values = df.values
    values = values.astype('float32')
    return values

In [212]:
#코드들로부터 값을 추출하고 합치기
def get_from_codes(codes):
    for j in codes:
        df1 = globals()['df_'+j]
        values = get_float_values(df1)
        reframed = series_to_supervised(values, 3, 1)
        reframed = delete_unuse_var(reframed)
        global df_all
        df_all = pd.concat([reframed,df_all])
    return df_all

In [213]:
def fill_nan(X_list,Y,df_a):
    lin_reg = linear_model.LinearRegression()
    x = df_a.dropna(axis=0)[X_list]
    y = df_a.dropna(axis=0)[Y]
    lin_reg_model = lin_reg.fit(x,y)
    y_pred = lin_reg_model.predict(df_a.loc[:, X_list])
    df_a[Y] = np.where(df_a[Y].isnull(),pd.Series(y_pred.flatten()),df_a[Y])
    return df_a

파일 읽기

In [214]:
df = pd.read_csv('final6.csv',encoding='cp949')

In [215]:
#결측값 제거
df = df[df['sale_type'].notna()]
df = df[df['parking'].notna()]
df = df[df['HAI'].notna()]
df = df[df['M2'].notna()]
#정성적변수 정량적 변수로 변환
df.loc[df['sale_type'] == '분양', 'sale_type'] = 0
df.loc[df['sale_type'] == '임대+분양', 'sale_type'] = 1
df.loc[df['sale_type'] == '임대', 'sale_type'] = 2
df.loc[df['sale_type'] == '영구임대', 'sale_type'] = 3
df['area_m3']=df['area_m3'].round(-1)
df = df.astype({'prices':'float64'})
df['prices']=df.groupby(['YYYYMM','apart_code','area_m3'])['prices'].transform('mean')
df['apart_code_area'] = df['apart_code'].map(str) +'_' +df['area_m3'].map(int).map(str)
df_a = df
exiting_code = df['apart_code'].unique()
df_vary = df.drop_duplicates(['apart_code'])
df_range = df_vary[['apart_code','GPS_X','GPS_Y']]
many_df = pd.read_csv('서울시아파트(기준).csv',encoding='cp949')
many_df = many_df[many_df['k-아파트코드'].isin(exiting_code)]

예측하고 싶은 값 추가

In [216]:
df_predction_8 = df.drop_duplicates(subset = ['apart_code','area_m3'])

In [217]:
df_predction_8['YYYYMM'] = 202208
df_predction_8['crime_rate'] = np.nan
df_predction_8['ASPI'] = np.nan
df_predction_8['YYYY'] = 2022
df_predction_8['MM'] = 8
df_predction_8['HAI'] = np.nan
df_predction_8['PIR'] = np.nan
df_predction_8['LIR'] = np.nan
df_predction_8['GDP'] = np.nan
df_predction_8['GDI'] = np.nan
df_predction_8['GNI'] = np.nan
df_predction_8['CPI'] = np.nan
df_predction_8['avg_interest'] = np.nan
df_predction_8['kospi'] = np.nan
df_predction_8['M2'] = np.nan
df_predction_8['house_CCSI'] = np.nan
df_predction_8['unsold'] = np.nan

c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [218]:
df_predction_8.keys()

Index(['apart_code', 'YYYYMM', 'apart__name', 'sigungu', 'town_name',
       'address', 'sale_type', 'area_m3', 'floor', 'const_year', 'parking',
       'GPS_X', 'GPS_Y', 'crime_rate', 'building_year', 'market_dist',
       'busstop_dist', 'subway_dist', 'kinder_dist', 'hospital_dist', 'ASPI',
       'YYYY', 'MM', 'HAI', 'PIR', 'LIR', 'GDP', 'GDI', 'GNI', 'CPI',
       'avg_interest', 'kospi', 'M2', 'house_CCSI', 'unsold', 'prices',
       'near_element', 'near_middle', 'near_high', 'generation', 'const',
       'const_rank', 'apart_code_area'],
      dtype='object')

In [219]:
#가격은 뭘로 할건지
df_predction_8['prices'] = 0 

c:\Users\46861\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [220]:
df_a['crime_rate']

0         1.116650
1         1.116650
2         1.116650
3         1.116650
4         1.116650
            ...   
725041    0.729671
725044    2.590410
725045    2.590410
725046    2.590410
725047    2.590410
Name: crime_rate, Length: 717264, dtype: float64

In [221]:
#df_prediction_8 과 df_all을 결합
df_a = pd.concat([df_a,df_predction_8], ignore_index=False)


In [223]:
df_b = df_a

In [225]:

df_a = df_b
df_a = fill_nan(['YYYYMM', 'YYYY', 'MM','GPS_X','GPS_Y'],['crime_rate'],df_a)
filling_list = ['ASPI','HAI', 'PIR', 'LIR', 'GDP', 'GDI', 'GNI', 'CPI','avg_interest', 'kospi', 'M2','house_CCSI']
for i in filling_list:
    df_a = fill_nan(['YYYYMM', 'YYYY', 'MM'],[i],df_a)
df_a = fill_nan(['YYYYMM', 'YYYY', 'MM'],['unsold'],df_a)

MemoryError: Unable to allocate 3.82 TiB for an array with shape (724270, 724270) and data type float64

실제로 돌아가는 모델

In [126]:
#6분정도 걸리는듯
#exiting_code = df['apart_code'].unique()
test_codes = random.sample(list(exiting_code),10)
for i in test_codes:
    gizun_code = i
    gizun = df_range[df_range['apart_code']==gizun_code]
    #변수 변경시 거리 조정
    near_df = finding_near_apt(gizun,2)
    near_df = pd.merge(near_df,many_df,left_on='apart_code',right_on='k-아파트코드',how='left')
    near_df = near_df[['apart_code','k-전체세대수']]
    gizun_count = near_df[near_df['apart_code']==gizun_code].iloc[0][1]
    indexNames = near_df[near_df['k-전체세대수'] < gizun_count].index
    near_df.drop(indexNames , inplace=True)
    df = df_a
    df = pd.merge(near_df,df,on='apart_code')
    df.pop('k-전체세대수')
    df = df.drop_duplicates(subset=['apart_code_area','YYYYMM'])
    df = df.sort_values(by=["apart_code_area", "YYYYMM"], ascending=[True, True])
    df = df[['apart_code_area', 'YYYYMM','sale_type', 'area_m3', 'floor', 'const_year', 
    'parking','crime_rate', 'building_year', 'market_dist','busstop_dist', 'subway_dist', 
    'kinder_dist', 'hospital_dist', 'ASPI','YYYY', 'MM', 'HAI', 'PIR', 'LIR', 'GDP', 'GDI', 
    'GNI', 'CPI','avg_interest', 'kospi', 'M2', 'house_CCSI', 'unsold','near_element', 
    'near_middle', 'near_high', 'generation','const_rank','prices']]
    codes = df['apart_code_area'].unique()
    groups = df.groupby(df.apart_code_area)
    for j in codes:
        globals()['df_'+j] = groups.get_group(j)
    gizun_code_areas = df[df['apart_code_area'].str.contains(gizun_code)]['apart_code_area'].unique()
    for k in gizun_code_areas:
        gizun_code_area = k
        gizun_df = globals()['df_'+gizun_code_area]
        df_all = pd.DataFrame()
        deleted_code = 'df_'+gizun_code_area
        codes = np.delete(codes, np.where(codes == deleted_code))
        df_all = get_from_codes(codes)
        train_set = df_all[df_all['var1(t)']<202208]
        df_test = gizun_df
        values = get_float_values(df_test)
        reframed = series_to_supervised(values, 3, 1)
        reframed = delete_unuse_var(reframed)
        #날짜를 사용해 df와 df_prediction_8 구분
        test_set = reframed[reframed['var1(t)']>=202208]
        df_set = pd.concat([train_set,test_set])
        train_days =  len(train_set)
        values= df_set.values
        scaled = scaler.fit_transform(values)
        values = scaled
        train_days =  len(train_set)
        #테스트 데이터 얼마나 사용할 건지
        train = values[:train_days]
        test = values[train_days:]
        # split into input and outputs
        train_X, train_y = train[:, :-1], train[:, -1]
        test_X, test_y = test[:, :-1], test[:, -1]
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
        model = Sequential()
        model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer='adam')
        history = model.fit(train_X, train_y, epochs=50, batch_size=72, 
                validation_data=(test_X, test_y), verbose=2, shuffle=False)
        # make a prediction
        try:
            yhat = model.predict(test_X)
            test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
            inv_yhat = concatenate((test_X[:, 0:],yhat), axis=1)
            # invert scaling for actual
            inv_yhat = scaler.inverse_transform(inv_yhat)
            inv_yhat = inv_yhat[:,-1]
            #만약 해당 값을 데이터 프레임에 넣고 싶다면
            #df_predction_8['prices'][df_predction_8['apart_code_area']==k] = inv_yhat
        except:
            #만약 해당 아파트의 2020년 이후 거래내역이 존재하지 않을 경우 오류 발생
            print('오류 발생') 
    

Epoch 1/50
118/118 - 3s - loss: 0.0693 - 3s/epoch - 24ms/step
Epoch 2/50
118/118 - 0s - loss: 0.0431 - 487ms/epoch - 4ms/step
Epoch 3/50
118/118 - 0s - loss: 0.0316 - 460ms/epoch - 4ms/step
Epoch 4/50
118/118 - 0s - loss: 0.0245 - 448ms/epoch - 4ms/step
Epoch 5/50
118/118 - 0s - loss: 0.0241 - 418ms/epoch - 4ms/step
Epoch 6/50
118/118 - 0s - loss: 0.0207 - 420ms/epoch - 4ms/step
Epoch 7/50
118/118 - 0s - loss: 0.0195 - 470ms/epoch - 4ms/step
Epoch 8/50
118/118 - 0s - loss: 0.0155 - 430ms/epoch - 4ms/step
Epoch 9/50
118/118 - 0s - loss: 0.0162 - 436ms/epoch - 4ms/step
Epoch 10/50
118/118 - 0s - loss: 0.0181 - 442ms/epoch - 4ms/step
Epoch 11/50
118/118 - 0s - loss: 0.0166 - 413ms/epoch - 3ms/step
Epoch 12/50
118/118 - 0s - loss: 0.0163 - 468ms/epoch - 4ms/step
Epoch 13/50
118/118 - 0s - loss: 0.0155 - 417ms/epoch - 4ms/step
Epoch 14/50
118/118 - 0s - loss: 0.0149 - 410ms/epoch - 3ms/step
Epoch 15/50
118/118 - 0s - loss: 0.0149 - 388ms/epoch - 3ms/step
Epoch 16/50
118/118 - 0s - loss: 0.0

KeyboardInterrupt: 

In [129]:
test_set

,var34(t-3),var34(t-2),var34(t-1),var1(t),var2(t),var3(t),var4(t),var5(t),var6(t),var7(t),...,var25(t),var26(t),var27(t),var28(t),var29(t),var30(t),var31(t),var32(t),var33(t),var34(t)


In [128]:
train_set

,var34(t-3),var34(t-2),var34(t-1),var1(t),var2(t),var3(t),var4(t),var5(t),var6(t),var7(t),...,var25(t),var26(t),var27(t),var28(t),var29(t),var30(t),var31(t),var32(t),var33(t),var34(t)
3,34200.000000,34075.000000,33916.667969,201204.0,0.0,80.0,16.0,2003.0,0.0,0.922525,...,1981.989990,1777114.750,106.599998,1691.0,0.17805,0.189964,0.191992,1344.0,4.0,34800.000000
4,34075.000000,33916.667969,34800.000000,201206.0,0.0,80.0,22.0,2003.0,0.0,0.922525,...,1854.010010,1796981.500,94.199997,1698.0,0.17805,0.189964,0.191992,1344.0,4.0,32066.666016
5,33916.667969,34800.000000,32066.666016,201207.0,0.0,80.0,9.0,2003.0,0.0,0.922525,...,1881.989990,1807289.250,97.800003,3146.0,0.17805,0.189964,0.191992,1344.0,4.0,33100.000000
6,34800.000000,32066.666016,33100.000000,201208.0,0.0,80.0,12.0,2003.0,0.0,0.922525,...,1905.119995,1817134.875,103.199997,3847.0,0.17805,0.189964,0.191992,1344.0,4.0,33200.000000
7,32066.666016,33100.000000,33200.000000,201209.0,0.0,80.0,4.0,2003.0,0.0,0.922525,...,1996.209961,1819290.125,108.300003,3754.0,0.17805,0.189964,0.191992,1344.0,4.0,31833.333984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,118540.000000,117000.000000,119700.000000,202109.0,1.0,60.0,21.0,2019.0,2610.0,0.578368,...,3068.820068,3513681.000,131.800003,55.0,0.44205,0.331741,0.445727,2352.0,4.0,122000.000000
13,117000.000000,119700.000000,122000.000000,202111.0,1.0,60.0,3.0,2019.0,2610.0,0.578368,...,2839.010010,3594723.000,111.599998,54.0,0.44205,0.331741,0.445727,2352.0,4.0,117000.000000
14,119700.000000,122000.000000,117000.000000,202112.0,1.0,60.0,25.0,2019.0,2610.0,0.578368,...,2977.649902,3620057.000,103.199997,54.0,0.44205,0.331741,0.445727,2352.0,4.0,120000.000000
15,122000.000000,117000.000000,120000.000000,202201.0,1.0,60.0,16.0,2019.0,2610.0,0.578368,...,2663.340088,3634342.000,101.400002,47.0,0.44205,0.331741,0.445727,2352.0,4.0,120000.000000


In [ ]:
df_predction_8

,apart_code,YYYYMM,apart__name,sigungu,town_name,address,sale_type,area_m3,floor,const_year,...,house_CCSI,unsold,prices,near_element,near_middle,near_high,generation,const,const_rank,apart_code_area
0,A10024240,202208,무지개,서울특별시 서초구 서초동,서초동,서울특별시 서초구 효령로 391,1,100.0,1,1978,...,NaN,NaN,NaN,0.241347,0.551754,0.886371,1446.0,지에스건설(주),4,A10024240_100
2,A10024240,202208,무지개,서울특별시 서초구 서초동,서초동,서울특별시 서초구 효령로 391,1,80.0,6,1978,...,NaN,NaN,NaN,0.241347,0.551754,0.886371,1446.0,지에스건설(주),4,A10024240_80
10,A10024240,202208,무지개,서울특별시 서초구 서초동,서초동,서울특별시 서초구 효령로 391,1,140.0,6,1978,...,NaN,NaN,NaN,0.241347,0.551754,0.886371,1446.0,지에스건설(주),4,A10024240_140
17,A10024240,202208,무지개,서울특별시 서초구 서초동,서초동,서울특별시 서초구 효령로 391,1,120.0,10,1978,...,NaN,NaN,NaN,0.241347,0.551754,0.886371,1446.0,지에스건설(주),4,A10024240_120
436,A10024603,202208,아이파크종암동2차,서울특별시 성북구 종암동,종암동,서울특별시 성북구 종암로9길 71,2,80.0,5,2005,...,NaN,NaN,NaN,0.530653,0.509385,0.761375,104.0,NaN,4,A10024603_80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725040,A10023991,202208,태릉해링턴플레이스,서울특별시 노원구 공릉동,공릉동,서울특별시 노원구 공릉로34길 86,1,60.0,12,2022,...,NaN,NaN,NaN,0.478162,0.465578,0.792782,1308.0,효성중공업,4,A10023991_60
725041,A10023991,202208,태릉해링턴플레이스,서울특별시 노원구 공릉동,공릉동,서울특별시 노원구 공릉로34길 86,1,70.0,19,2022,...,NaN,NaN,NaN,0.478162,0.465578,0.792782,1308.0,효성중공업,4,A10023991_70
725044,A10024359,202208,삼정아트테라스정동,서울특별시 중구 정동,정동,서울특별시 중구 서소문로 89-20,0,30.0,2,2020,...,NaN,NaN,NaN,0.177679,0.349024,0.260624,168.0,삼정기업,4,A10024359_30
725045,A10024359,202208,삼정아트테라스정동,서울특별시 중구 정동,정동,서울특별시 중구 서소문로 89-20,0,10.0,8,2020,...,NaN,NaN,NaN,0.177679,0.349024,0.260624,168.0,삼정기업,4,A10024359_10


In [ ]:
df_predction_8['prices'].isna().sum()

7006